# DNN Classifier

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report

** Get the Data **

In [15]:
df = pd.read_csv("dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 19 columns):
CAX_ID      314 non-null object
Y           234 non-null float64
flag        314 non-null object
Factor1     314 non-null float64
Factor2     314 non-null float64
Factor3     314 non-null float64
Factor4     314 non-null float64
Factor5     314 non-null float64
Factor6     314 non-null float64
Factor7     314 non-null float64
Factor8     314 non-null float64
Factor9     314 non-null float64
Factor10    314 non-null float64
Factor11    314 non-null float64
Factor12    314 non-null float64
Factor13    314 non-null float64
Factor14    314 non-null float64
Factor15    314 non-null float64
Factor16    314 non-null float64
dtypes: float64(17), object(2)
memory usage: 46.7+ KB


** Train Test Split **

In [16]:
train = df.loc[df['flag']=='TRAIN'].copy()
test = df.loc[df['flag']=='TEST'].copy()

X_train = train.drop(['CAX_ID','Y','flag'],axis=1)
X_test = test.drop(['CAX_ID','Y','flag'],axis=1)
y_train = train['Y']
y_test = test['Y']

** Continuous feature columns **

In [17]:
Factor1 = tf.feature_column.numeric_column("Factor1")
Factor2 = tf.feature_column.numeric_column("Factor2")
Factor3 = tf.feature_column.numeric_column("Factor3")
Factor4 = tf.feature_column.numeric_column("Factor4")
Factor5 = tf.feature_column.numeric_column("Factor5")
Factor6 = tf.feature_column.numeric_column("Factor6")
Factor7 = tf.feature_column.numeric_column("Factor7")
Factor8 = tf.feature_column.numeric_column("Factor8")
Factor9 = tf.feature_column.numeric_column("Factor9")
Factor10 = tf.feature_column.numeric_column("Factor10")
Factor11 = tf.feature_column.numeric_column("Factor11")
Factor12 = tf.feature_column.numeric_column("Factor12")
Factor13 = tf.feature_column.numeric_column("Factor13")
Factor14 = tf.feature_column.numeric_column("Factor14")
Factor15 = tf.feature_column.numeric_column("Factor15")
Factor16 = tf.feature_column.numeric_column("Factor16")

In [18]:
feat_cols = [Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,
             Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15,Factor16]

** Input Function **

In [19]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=80,num_epochs=1000,shuffle=True)

** Model with tf.estimator **

In [20]:
model = tf.estimator.DNNClassifier(hidden_units=[32,24,16,8,16],feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_session_config': None, '_model_dir': 'C:\\Users\\admin\\AppData\\Local\\Temp\\tmp73bskdre'}


** Train the model for at 2000 steps **

In [21]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\admin\AppData\Local\Temp\tmp73bskdre\model.ckpt.
INFO:tensorflow:step = 1, loss = 55.7754
INFO:tensorflow:global_step/sec: 117.889
INFO:tensorflow:step = 101, loss = 2.27417 (0.842 sec)
INFO:tensorflow:global_step/sec: 121.838
INFO:tensorflow:step = 201, loss = 0.058807 (0.824 sec)
INFO:tensorflow:global_step/sec: 129.03
INFO:tensorflow:step = 301, loss = 0.0217847 (0.778 sec)
INFO:tensorflow:global_step/sec: 134.251
INFO:tensorflow:step = 401, loss = 0.00642878 (0.751 sec)
INFO:tensorflow:global_step/sec: 127.401
INFO:tensorflow:step = 501, loss = 0.0112889 (0.757 sec)
INFO:tensorflow:global_step/sec: 131.717
INFO:tensorflow:step = 601, loss = 0.0106996 (0.785 sec)
INFO:tensorflow:global_step/sec: 82.3587
INFO:tensorflow:step = 701, loss = 0.00334317 (1.204 sec)
INFO:tensorflow:global_step/sec: 93.3722
INFO:tensorflow:step = 801, loss = 0.00166281 (1.094 sec)
INFO:tensorflow:global_step/

** Prediction **

In [22]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [23]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Restoring parameters from C:\Users\admin\AppData\Local\Temp\tmp73bskdre\model.ckpt-2925


In [24]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [25]:
test_calc = pd.concat([pd.DataFrame(test).reset_index(drop=True),pd.DataFrame(final_preds).reset_index(drop=True)],axis=1)
test_calc.rename(columns={0: 'predicted'}, inplace=True)

In [26]:
test_calc.to_csv('test_calc.csv')